In [1]:
# Import all the things

import json
import pandas as pd
import numpy as np
import wrangle
import acquire
from prepare import basic_clean, tokenize, lemmatize, stem, remove_stopwords, prep_string_data#, split_data

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from wordcloud import WordCloud
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import nltk

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
# use our own wrangle to get the data
df = wrangle.get_npr_data()
df.head()

The df has 1889857 rows and 12 columns.


,story_id_num,utterance_order,speaker,utterance,program,title,is_host,clean,lemmatized,vader,date
0,57264,9,ms. loren mooney,"It's a 2,200-mile race. To give some sense of ...",talk of the nation,how to watch the tour de france,False,"it s a 2,200 mile race. to give some sense of ...","it s a 2,200 mile race. to give some sense of ...",0.0000,2010-07-12
1,57264,10,ms. loren mooney,So for a top competitor like Lance to try to m...,talk of the nation,how to watch the tour de france,False,so for a top competitor like lance to try to m...,so for a top competitor like lance to try to m...,0.9346,2010-07-12
2,57264,11,neal conan,"So in every team, presumably there's one star,...",talk of the nation,how to watch the tour de france,True,"so in every team , presumably there s one star...","so in every team , presumably there s one star...",0.7096,2010-07-12
3,57264,12,ms. loren mooney,That's right. Each team has nine riders. And w...,talk of the nation,how to watch the tour de france,False,that s right. each team has nine riders. and w...,that s right. each team ha nine riders. and wh...,0.9274,2010-07-12
4,57264,13,neal conan,"So slipstream, this is like drafting in car ra...",talk of the nation,how to watch the tour de france,True,"so slipstream , this is like drafting in car r...","so slipstream , this is like drafting in car r...",0.3612,2010-07-12


### I'm interested in topic modeling on a story-level basis
### So I will need to groupby story_id_num

In [5]:
df = df.sort_values(by=['story_id_num','utterance_order'])
df.head()

,story_id_num,utterance_order,speaker,utterance,program,title,is_host,clean,lemmatized,vader,date
13231,17,1,melissa block,He is in cahoots with a foreign government to ...,weekend edition sunday,n.j. democrat on reported whistleblower compla...,True,he is in cahoots with a foreign government to ...,he is in cahoot with a foreign government to m...,-0.8176,2019-09-22
13232,17,2,melissa block,Let's hear now from another Democrat. Tom Mali...,weekend edition sunday,n.j. democrat on reported whistleblower compla...,True,let s hear now from another democrat. tom mali...,let s hear now from another democrat. tom mali...,0.4404,2019-09-22
13233,17,3,tom malinowski,Good morning.,weekend edition sunday,n.j. democrat on reported whistleblower compla...,False,good morning .,good morning .,0.4404,2019-09-22
13234,17,4,melissa block,I'd like first to get your response to the Tru...,weekend edition sunday,n.j. democrat on reported whistleblower compla...,True,i d like first to get your response to the tru...,i d like first to get your response to the tru...,0.3612,2019-09-22
13235,17,5,tom malinowski,"Well, first of all, it's illegal. We have a ve...",weekend edition sunday,n.j. democrat on reported whistleblower compla...,False,"well , first of all , it s illegal. we have a ...","well , first of all , it s illegal. we have a ...",0.8680,2019-09-22


In [8]:
df = df[['story_id_num','utterance_order','title','lemmatized']]

In [9]:
df_sample = df[:100]
df_sample.head()

,story_id_num,utterance_order,title,lemmatized
13231,17,1,n.j. democrat on reported whistleblower compla...,he is in cahoot with a foreign government to m...
13232,17,2,n.j. democrat on reported whistleblower compla...,let s hear now from another democrat. tom mali...
13233,17,3,n.j. democrat on reported whistleblower compla...,good morning .
13234,17,4,n.j. democrat on reported whistleblower compla...,i d like first to get your response to the tru...
13235,17,5,n.j. democrat on reported whistleblower compla...,"well , first of all , it s illegal. we have a ..."


In [10]:
df_sample.shape

(100, 4)

In [16]:
df_sample = df_sample.sort_values(by=['story_id_num','utterance_order'])
df_sample.head()

,story_id_num,utterance_order,title,lemmatized
13231,17,1,n.j. democrat on reported whistleblower compla...,he is in cahoot with a foreign government to m...
13232,17,2,n.j. democrat on reported whistleblower compla...,let s hear now from another democrat. tom mali...
13233,17,3,n.j. democrat on reported whistleblower compla...,good morning .
13234,17,4,n.j. democrat on reported whistleblower compla...,i d like first to get your response to the tru...
13235,17,5,n.j. democrat on reported whistleblower compla...,"well , first of all , it s illegal. we have a ..."


In [32]:
# group every story as an aggregation of all its utterances
df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join)#.iloc[0]

story_id_num
17    he is in cahoot with a foreign government to m...
18    come with concrete plan , not beautiful speech...
19    if you stand outside the old dekalb county cou...
20    you may have spotted pink salt in your superma...
21    singing in foreign language . the other day , ...
Name: lemmatized, dtype: object

In [43]:
# group every story as an aggregation of all its utterances and store in a DF
stories_test = pd.DataFrame(df_sample.groupby(['story_id_num']).lemmatized.agg(' '.join))
stories_test

,lemmatized
story_id_num,
17,he is in cahoot with a foreign government to m...
18,"come with concrete plan , not beautiful speech..."
19,if you stand outside the old dekalb county cou...
20,you may have spotted pink salt in your superma...
21,"singing in foreign language . the other day , ..."


In [42]:
# stories.iloc[0].values

### Run the above on the whole df

In [47]:
df = df.sort_values(by=['story_id_num','utterance_order'])
df.head()

,story_id_num,utterance_order,title,lemmatized
13231,17,1,n.j. democrat on reported whistleblower compla...,he is in cahoot with a foreign government to m...
13232,17,2,n.j. democrat on reported whistleblower compla...,let s hear now from another democrat. tom mali...
13233,17,3,n.j. democrat on reported whistleblower compla...,good morning .
13234,17,4,n.j. democrat on reported whistleblower compla...,i d like first to get your response to the tru...
13235,17,5,n.j. democrat on reported whistleblower compla...,"well , first of all , it s illegal. we have a ..."


In [53]:
df.dtypes

story_id_num        int64
utterance_order     int64
title              object
lemmatized         object
dtype: object

In [54]:
df.isna().sum()

story_id_num       0
utterance_order    0
title              0
lemmatized         2
dtype: int64

In [55]:
df = df.dropna()

#### well...it worked on the sample above
#### had to drop 2 nulls that existed in the lemmatized column

In [56]:
# group every story as an aggregation of all its utterances and store in a DF
stories = pd.DataFrame(df.groupby(['story_id_num']).lemmatized.agg(' '.join))
stories

,lemmatized
story_id_num,
17,he is in cahoot with a foreign government to m...
18,"come with concrete plan , not beautiful speech..."
19,if you stand outside the old dekalb county cou...
20,you may have spotted pink salt in your superma...
21,"singing in foreign language . the other day , ..."
...,...
140737,this is all thing considered from npr news. i ...
140738,"for low income family , coming up with all of ..."
140739,"from npr news , this is all thing considered. ..."
